In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

# Initial Ingestion and Cleaning

In [ ]:
# Loading the data from the JSON files
offers = pd.read_json('data/portfolio.json', orient='records', lines=True)
users = pd.read_json('data/profile.json', orient='records', lines=True)
interactions = pd.read_json('data/transcript.json', orient='records', lines=True)

# Mappings for renaming columns to keep better track of where the columns come from
offer_col_mapping = {
    'id': 'offer_id', 
    'channels': 'offer_notification_channel', 
    'offer_type': 'offer_type', 
    'difficulty': 'offer_spend_minimum', 
    'reward': 'offer_reward', 
    'duration': 'offer_duration'
}

users_col_mapping = {
    'id': 'user_id', 
    'gender': 'user_gender', 
    'age': 'user_age', 
    'became_member_on': 'user_member_since', 
    'income': 'user_income'
}

interactions_col_mapping = {
    'person': 'user_id', 
    'event': 'intxn_event_type', 
    'value': 'intxn_value', 
    'time': 'intxn_time'
}

# Reordering and renaming columns for offers
offers = offers[ list( offer_col_mapping.keys() ) ]
offers = offers.rename( columns=offer_col_mapping )
offers['offer_duration'] = offers['offer_duration'].apply( lambda x: x * 24 ) # Converting days to hours
offers['offer_reward_vs_spend'] = round( offers['offer_reward'] / offers['offer_spend_minimum'], 2 )
offers['offer_reward_vs_spend'] = offers['offer_reward_vs_spend'].fillna(0)

# Exploding the offers table so that we have one row per channel per offer
offers = offers.explode('offer_notification_channel')

# Reordering and renaming columns for users
users = users[ list( users_col_mapping.keys() ) ]
users = users.rename( columns=users_col_mapping )

# Users with age 118 are users that do not have an age in their profile
users['user_gender'] = users['user_gender'].fillna('N/A')

# Creating the user tenure column from the user_member_since column
users['user_member_since'] = pd.to_datetime(users['user_member_since'], format='%Y%m%d', errors='coerce')
users['user_tenure'] = users['user_member_since'].apply( lambda x: ( pd.Timestamp(2019, 1, 1) - x).days if pd.notnull(x) else np.nan )
users['user_tenure'] = users['user_tenure'].apply( lambda x: int(x / 360) )

# Assigning a user age group
users['user_age'] = users['user_age'].apply( lambda x: np.nan if x == 118 else x )
users['user_age_group'] = pd.cut( users['user_age'], bins=[0, 18, 25, 35, 45, 55, 65, np.inf], labels=['0-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+'], right=False )
users['user_age_group'] = users['user_age_group'].cat.add_categories("N/A").fillna("N/A")

# Assigning a user income group
users['user_income_group'] = pd.cut( users['user_income'], bins=[0, 50000, 75000, 100000, np.inf], labels=['<50k', '50k-75k', '75k-100k', '>100k'], right=False )
users['user_income_group'] = users['user_income_group'].cat.add_categories("N/A").fillna("N/A")

# Reordering and renaming columns for interactions
interactions = interactions[ list( interactions_col_mapping.keys() ) ]
interactions = interactions.rename( columns=interactions_col_mapping )

# Expanding the interaction value column into separate columns
intxn_value_expanded = pd.json_normalize( interactions['intxn_value'] )

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html
# Coalescing the two offer id columns into one since one of them has a space in the name
intxn_value_expanded['offer_id'] = intxn_value_expanded['offer_id'].combine_first( intxn_value_expanded['offer id'] )
intxn_value_expanded = intxn_value_expanded.drop(columns=['offer id'])
intxn_value_expanded = intxn_value_expanded.rename( columns={ col: f'intxn_{col}' for col in intxn_value_expanded.columns.tolist() if col != 'offer_id' } )

# Adding the expanded interaction value columns back into the interactions dataframe
interactions = pd.concat([interactions, intxn_value_expanded], axis=1)
interactions = interactions.drop(columns=['intxn_value'])

# Renaming the columns for the interactions dataframe
interactions_cols = ['user_id', 'offer_id']
interactions_cols += [ col for col in interactions.columns if col.startswith('intxn_') ]
interactions = interactions[ interactions_cols ]

# Loading into Database

In [ ]:
conn = create_engine('sqlite:///data/starbucks_data.db')

offers.to_sql('offers', conn, if_exists='replace', index=False)
users.to_sql('users', conn, if_exists='replace', index=False)
interactions.to_sql('interactions', conn, if_exists='replace', index=False)

print("Data loaded into SQLite database successfully.")

306534